In [1]:
import numpy as np
import pandas as pd 

In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
d1= dict(np.load("/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/xla/default/train/alexnet_train_batch_32.npz"))

In [5]:
d1

{'edge_index': array([[  2,   0],
        [  2,   1],
        [  5,   3],
        ...,
        [371, 366],
        [371, 367],
        [371, 370]]),
 'node_feat': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'node_opcode': array([ 63,  63,  57,  63,  63,  57,  63,  63,  57,  63,  63,  57,  63,
         63,   2,  63,  63,   2,  63,  63,  20,  63,  63,   2,  63,  63,
         20,  63,  63,   2,  63,  63,  20,  63,  63,   2,  63,  63,   2,
         63,  63,   2,  63,  63,   2,  63,  63,   2,  63,  63,   2,  63,
         63,   2,  63,  63,   2,  63,  63,   2,  63,  45,  24,   2,  45,
         47,  45,  45,  13,  48,  20,  24,  13,  24,  13,  83,  60,  20,
         77,  24,  13,  20,  77,  20,  45,  45,  26,  45,  13,   2,  13,
         57,  24,  72,  45,  26,  45,  13,   2, 

In [6]:
d_tile=dict(np.load("/kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla/train/alexnet_train_batch_32_-1bae27a41d70f4dc.npz"))

In [7]:
d_tile

{'node_feat': array([[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'node_opcode': array([63, 11, 63, 11, 63, 41, 63, 41, 26, 63, 63, 41], dtype=uint8),
 'edge_index': array([[ 1,  0],
        [ 3,  2],
        [ 5,  1],
        [ 5,  4],
        [ 7,  3],
        [ 7,  6],
        [ 8,  5],
        [ 8,  7],
        [11,  8],
        [11,  9],
        [11, 10]]),
 'config_feat': array([[ 32.,  32.,   0., ...,   0.,   0.,   0.],
        [ 32.,   1.,   0., ...,   0.,   0.,   0.],
        [ 16.,   4.,   0., ...,   0.,   0.,   0.],
        ...,
        [144.,   7.,   0., ...,   0.,   0.,   0.],
        [160.,   8.,   0., ...,   0.,   0.,   0.],
        [160.,   8.,   0., ...,   0.,   0.,   0.]], dtype=float32),
 'config_runtime': array([ 263238, 2029255, 1192602, 1027600, 1962135,  

In [8]:
import torch
from torch import Tensor
from torch_geometric.nn import GCNConv
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

In [9]:
t_f=torch.from_numpy(d_tile["config_feat"])

In [10]:
t_e=torch.from_numpy(d_tile["edge_index"])

In [11]:
print(t_e.shape)
print(t_f.shape)

torch.Size([11, 2])
torch.Size([266, 24])


In [12]:
t_e=torch.transpose(t_e, 0, 1)

In [13]:
process=len(d_tile["config_runtime"])
model = GCN(24,16,266)

In [14]:
def replacetensor(tensor,process):
    dup=tensor.clone()
    dup=dup.detach().numpy()
    dup=dup.tolist()
    tensor,indices=torch.sort(tensor)
    i=0
    ind=dup.index(tensor[0])
    dup[ind]=i
    for j in range(1,process):
        ind=dup.index(tensor[j])
        if tensor[j-1]<tensor[j]:
            i+=1
        dup[ind]=i
        dup[ind]=int(dup[ind])
    dup=np.array(dup)
    dup=torch.from_numpy(dup)
    dup=dup.type(torch.LongTensor)
    return dup

Since features determine the compile sequence to identify datatypes and dimensions of vectors, the gnn built would use features as centers to determine the sequence of operations, which would we compared against the config runtime. The maximum subsequence of the graph nn with least runtime would be returned. Opcodes could be thought to similar to the bag-of-words in cora that determine the relative increase or decrease of runtime. The classes of runtime would be 'c' distinct or similar runtimes ordered in increasing order.

Use the layout features to link to the configurable parts of the processes set up to recompute the order of the best runtimes obtained.For example, if some non configurable sequence is found to run for a lesser time in the tile's confguration, recheck with the layout's reconfigurable means. Mark the activated features of each node to calculate a runtime overhead and interconnections of input acceptance order.

In [15]:
d_test=dict(np.load("/kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla/test/0023795810403f8b0b244d88c901322f.npz"))

In [16]:
def weights():
    weights=[0.0055,0]
    for i in range(2,21):
        weights.append(0.005)
    for i in range(21,31):
        weights.append(0.006)
    for i in range(31,37):
        weights.append(0.0065)
    for i in range(37,85):
        weights.append(0.00675)
    for i in range(85,93):
        weights.append(0.0068)
    weights.append(0.0065)
    weights.append(0.0065)
    for i in range(95,140):
        weights.append(0.01)
    return weights
    
    

Add the runtimes of all the nodes subject to their initial configuration to obtain the estimated tile runtime.Obtain the standard deviation of the given configuration runtimes to have the probabilities of obtaining the top configurations.

In [17]:
def runtimepernode(weight,feature_vector):
    lf=feature_vector.shape
    rv=[]
    for i in range(lf[0]):
        rt=0
        for j in range(lf[1]):
            rt+=weight[j]*feature_vector[i][j]
        rv.append(rt)
    return rv

In [18]:
def addruntime(edges_vector,cumruntime):
    le=len(edges_vector)
    fr=cumruntime
    fr[edges_vector[0][0]]=(cumruntime[edges_vector[0][0]]+cumruntime[edges_vector[0][1]])
    for i in range(1,le):
        if cumruntime[edges_vector[i-1][0]]!=cumruntime[edges_vector[i][0]]:
            fr[edges_vector[i][0]]=(cumruntime[edges_vector[i][0]]+cumruntime[edges_vector[i][1]])
        else:
            fr[edges_vector[i][0]]+=cumruntime[edges_vector[i][1]]
    return fr

In [19]:
wt=weights()
over_run=runtimepernode(wt,d_tile["node_feat"])
final_run=addruntime(d_tile["edge_index"],over_run)

In [20]:
total_run=sum(final_run)

In [21]:
runtime=torch.from_numpy((d_tile["config_runtime"]-total_run)/d_tile["config_runtime_normalizers"])
runtime=replacetensor(runtime,process)

In [22]:
import torch.nn as nn
import torch.nn.functional as F
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
for epoch in range(100):
    pred = model(t_f,t_e)
    loss = F.cross_entropy(pred,runtime)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss)

tensor(70.2462, grad_fn=<NllLossBackward0>)
tensor(57.2592, grad_fn=<NllLossBackward0>)
tensor(48.3442, grad_fn=<NllLossBackward0>)
tensor(40.7255, grad_fn=<NllLossBackward0>)
tensor(35.8239, grad_fn=<NllLossBackward0>)
tensor(31.1329, grad_fn=<NllLossBackward0>)
tensor(27.3662, grad_fn=<NllLossBackward0>)
tensor(24.4015, grad_fn=<NllLossBackward0>)
tensor(21.6788, grad_fn=<NllLossBackward0>)
tensor(19.6505, grad_fn=<NllLossBackward0>)
tensor(17.8433, grad_fn=<NllLossBackward0>)
tensor(16.3877, grad_fn=<NllLossBackward0>)
tensor(15.2194, grad_fn=<NllLossBackward0>)
tensor(14.5212, grad_fn=<NllLossBackward0>)
tensor(13.8579, grad_fn=<NllLossBackward0>)
tensor(13.0723, grad_fn=<NllLossBackward0>)
tensor(12.1841, grad_fn=<NllLossBackward0>)
tensor(11.3946, grad_fn=<NllLossBackward0>)
tensor(10.7562, grad_fn=<NllLossBackward0>)
tensor(10.2193, grad_fn=<NllLossBackward0>)
tensor(9.7521, grad_fn=<NllLossBackward0>)
tensor(9.2692, grad_fn=<NllLossBackward0>)
tensor(8.7821, grad_fn=<NllLossBac

In [23]:
pred

tensor([[ 2.3596,  2.6858,  2.7538,  ..., -0.1456,  2.1677, -2.0213],
        [-4.3004, -3.8714, -3.5723,  ..., -0.4074,  2.8662,  1.4173],
        [-5.9020, -5.9789, -4.3509,  ...,  0.4430,  1.3440,  1.8558],
        ...,
        [ 4.1754,  3.3224,  3.7602,  ...,  2.2905,  0.0521, -1.2616],
        [15.8548, 14.5631, 15.9710,  ...,  2.1153,  2.8480, -7.9370],
        [11.3168, 10.4056, 10.9601,  ...,  3.2121,  1.3120, -5.4481]],
       grad_fn=<AddBackward0>)

In [24]:
torch.save(model.state_dict(),"tilerun.pt")

In [25]:
ds=d1["node_config_feat"].shape

Consider each node configuration id to be a tile each. Predict the runtime for each tile with default configuration, and compare with respect to given runtimes of each layout. Multiply each node of the second vector with the runtime obtained for a node to run and reduce the dimension to proceed with the steps similar to deduce the tile configuration's probabilities.

In [26]:
over_run_layout=runtimepernode(wt,d1["node_feat"])
final_run_layout=addruntime(d1["edge_index"],over_run_layout)

In [27]:
def layoutconfig(final_run,config_ids,config_vector):
    lc=len(config_ids)
    cs=config_vector.shape
    new_cv=[]
    mult=[]
    for i in range(cs[0]):
        for j in range(lc):
            mult=(config_vector[i][j]*final_run[config_ids[j]]).tolist()
        new_cv.append(mult)
    return new_cv

In [28]:
new_con=layoutconfig(final_run_layout,d1["node_config_ids"],d1["node_config_feat"][:4000,:,:])

In [29]:
total_run1=sum(final_run_layout)

In [30]:
runtimel=torch.from_numpy((d1["config_runtime"][:4000]-total_run1))

In [31]:
runtimeln=replacetensor(runtimel,4000)

In [32]:
torch.save(runtimeln,"runtimelayout.pt")

In [33]:
new_conn=np.array(new_con)

In [34]:
t_fl=torch.from_numpy(new_conn)

In [35]:
model1=GCN(ds[2],16,4000)

In [36]:
t_fl=t_fl.float()

In [37]:
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.01)
for epoch in range(100):
    pred1 = model1(t_fl,t_e)
    loss1 = F.cross_entropy(pred1,runtimeln)
    optimizer1.zero_grad()
    loss1.backward()
    optimizer1.step()
    print(loss1)

tensor(475394.6875, grad_fn=<NllLossBackward0>)
tensor(396094.0625, grad_fn=<NllLossBackward0>)
tensor(313593.1562, grad_fn=<NllLossBackward0>)
tensor(246938.7031, grad_fn=<NllLossBackward0>)
tensor(179846.5312, grad_fn=<NllLossBackward0>)
tensor(120358.0938, grad_fn=<NllLossBackward0>)
tensor(82834.3984, grad_fn=<NllLossBackward0>)
tensor(55787.4219, grad_fn=<NllLossBackward0>)
tensor(32973.7695, grad_fn=<NllLossBackward0>)
tensor(10024.3477, grad_fn=<NllLossBackward0>)
tensor(8.2926, grad_fn=<NllLossBackward0>)
tensor(8.2924, grad_fn=<NllLossBackward0>)
tensor(8.2922, grad_fn=<NllLossBackward0>)
tensor(8.2920, grad_fn=<NllLossBackward0>)
tensor(8.2917, grad_fn=<NllLossBackward0>)
tensor(8.2915, grad_fn=<NllLossBackward0>)
tensor(8.2913, grad_fn=<NllLossBackward0>)
tensor(8.2910, grad_fn=<NllLossBackward0>)
tensor(8.2908, grad_fn=<NllLossBackward0>)
tensor(8.2906, grad_fn=<NllLossBackward0>)
tensor(8.2903, grad_fn=<NllLossBackward0>)
tensor(8.2901, grad_fn=<NllLossBackward0>)
tensor(8

In [38]:
print(model)

GCN(
  (conv1): GCNConv(24, 16)
  (conv2): GCNConv(16, 266)
)
